# Exponential Smoothing: Julia 
First experiments 

In [1]:
# import packages 
import Pkg
Pkg.add("CSV")
Pkg.add("DataFrames")
Pkg.add("BenchmarkTools")

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`


___
___
## Dataset Wind Turbine: Wind Speed

https://www.kaggle.com/datasets/berkerisen/wind-turbine-scada-dataset

In [25]:
#read in the time series data 
using CSV
using DataFrames

df = CSV.read("/Users/niklas/Documents/GitHub/Uni/10_Masterarbeit/data_files/wind_turbine_scada.csv", DataFrame)

wind_speed = df[!, 3]
wind_speed = filter(!ismissing, wind_speed)

# Replication factor
factor = 500

# Replicate the wind_speed array
wind_speed = repeat(wind_speed, factor)

println("Dataframe wind_speed has ", size(wind_speed, 1), " rows.")

Dataframe wind_speed has 25265000 rows.


___
### #1 Basic For Loop

In [3]:
using Statistics

# Julia array with wind_speed data
wind_speed = Float64.(wind_speed)

# Alpha is the smoothing factor.
alpha = 0.7

# List to store execution times
number_of_executions = 10
execution_times = []
smoothed_value = wind_speed[1]

# Perform exponential smoothing on the data
for i in 1:number_of_executions
    start_time = time()

    # Initialize the smoothed value with the first data point
    smoothed_value = wind_speed[1]

    # Perform exponential smoothing on the data
    for j in 2:length(wind_speed)
        smoothed_value = alpha * wind_speed[j] + (1 - alpha) * smoothed_value
    end

    # Stop the timer
    end_time = time()
    push!(execution_times, end_time - start_time)
end

# Calculate the function time
function_time = median(execution_times)

# Print the results
println("### Dataset wind_speed")
println("### #1 Basic For Loop \n")
println("The last smoothed value for wind_speed is: ", smoothed_value)
println("The function was executed in ", function_time, " seconds")


### Dataset wind_speed
### #1 Basic For Loop 

The last smoothed value for wind_speed is: 9.729056156830051
The function was executed in 5.656255006790161 seconds


In [27]:
using Statistics

@show Threads.nthreads()

# Julia array with wind_speed data
wind_speed = Float64.(wind_speed)

# Alpha is the smoothing factor.
alpha = 0.7

# List to store execution times
number_of_executions = 10
execution_times = []
smoothed_value = wind_speed[1]

# Perform exponential smoothing on the data
for i in 1:number_of_executions
    start_time = time()

    # Initialize the smoothed value with the first data point
    smoothed_value = wind_speed[1]

    # Perform exponential smoothing on the data
    for j in 2:length(wind_speed)
        smoothed_value = alpha * wind_speed[j] + (1 - alpha) * smoothed_value
    end

    # Stop the timer
    end_time = time()
    push!(execution_times, end_time - start_time)
end

# Calculate the function time
function_time = median(execution_times)

# Print the results
println("### Dataset wind_speed")
println("### #1 Basic For Loop \n")
println("The last smoothed value for wind_speed is: ", smoothed_value)
println("The function was executed in ", function_time, " seconds")


Threads.nthreads() = 1
### Dataset wind_speed
### #1 Basic For Loop 

The last smoothed value for wind_speed is: 9.729056156830051
The function was executed in 7.895835041999817 seconds


___
### #2 Vectorized

In [4]:
using Statistics

@show Threads.nthreads()

# Julia array with wind_speed data
wind_speed = Float64.(wind_speed)

# Alpha is the smoothing factor.
alpha = 0.7

# List to store execution times
number_of_executions = 10
execution_times = []
smoothed_value = wind_speed[1]

# Perform exponential smoothing on the data
for i in 1:number_of_executions
    start_time = time()

    # Perform exponential smoothing
    weights = alpha .* (1 - alpha) .^ (length(wind_speed)-1:-1:0)
    smoothed_values = cumsum(weights .* wind_speed) ./ cumsum(weights)
    smoothed_value = smoothed_values[end]

    # Stop the timer
    end_time = time()
    push!(execution_times, end_time - start_time)
end

# Calculate the function time
function_time = median(execution_times)

# Print the results
println("### Dataset wind_speed")
println("### #2 Vectorized \n")
println("The last smoothed value for wind_speed is: ", smoothed_value)
println("The function was executed in ", function_time, " seconds")


### Dataset wind_speed
### #2 Vectorized 

The last smoothed value for wind_speed is: 9.729056156830051
The function was executed in 2.58907949924469 seconds


___
### #3 Parallelized Basic For Loop

In [5]:
using Statistics

Threads.nthreads() = 8
@show Threads.nthreads()

# Julia array with wind_speed data
wind_speed = Float64.(wind_speed)

# Alpha is the smoothing factor.
alpha = 0.7

# List to store execution times
number_of_executions = 10
execution_times = []
smoothed_value = wind_speed[1]

# Perform exponential smoothing on the data
for i in 1:number_of_executions

    # Initialize the smoothed value with the first data point
    smoothed_value = wind_speed[1]

    # Create an array to hold the intermediate smoothed values for each thread
    smoothed_values = fill(smoothed_value, length(wind_speed))

    start_time = time()

    # Perform exponential smoothing on the data
    Threads.@threads for j in 2:length(wind_speed)
        smoothed_values[j] = alpha * wind_speed[j] + (1 - alpha) * smoothed_values[j-1]
    end

    # The last smoothed value is the last element of the smoothed_values array
    smoothed_value = smoothed_values[end]

    # Stop the timer
    end_time = time()
    push!(execution_times, end_time - start_time)
end

# Calculate the function time
function_time = median(execution_times)

# Print the results
println("### Dataset wind_speed")
println("### #3 Parallelized Basic For Loop \n")
println("The last smoothed value for wind_speed is: ", smoothed_value)
println("The function was executed in ", function_time, " seconds")


### Dataset wind_speed
### #3 Parallelized Basic For Loop 

The last smoothed value for wind_speed is: 9.729056156830051
The function was executed in 4.914914011955261 seconds


___
### #4 Parallelized Vectorized

In [31]:
using Statistics

Threads.nthreads() = 8
@show Threads.nthreads()

# Julia array with wind_speed data
wind_speed = Float64.(wind_speed)

# Alpha is the smoothing factor.
alpha = 0.7

# List to store execution times
number_of_executions = 10
execution_times = []
smoothed_value = wind_speed[1]

# Perform exponential smoothing on the data
for i in 1:number_of_executions
    start_time = time()

    # Perform exponential smoothing
    weights = alpha .* (1 - alpha) .^ (length(wind_speed)-1:-1:0)
    smoothed_values = cumsum(weights .* wind_speed) ./ cumsum(weights)
    smoothed_value = smoothed_values[end]

    # Stop the timer
    end_time = time()
    push!(execution_times, end_time - start_time)
end

# Calculate the function time
function_time = median(execution_times)

# Print the results
println("### Dataset wind_speed")
println("### #4 Vectorized \n")
println("The last smoothed value for wind_speed is: ", smoothed_value)
println("The function was executed in ", function_time, " seconds")

### Dataset wind_speed
### #4 Vectorized 

The last smoothed value for wind_speed is: 9.729056156830051
The function was executed in 2.8790249824523926 seconds


___
___
## Dataset: Solar Energy Production

In [7]:
#read in the time series data 
using CSV
using DataFrames

df = CSV.read("/Users/niklas/Documents/GitHub/Uni/10_Masterarbeit/data_files/energy_dataset.csv", DataFrame, missingstring=" ", silencewarnings=true, types=Dict(19 => Float64))

column_generation_solar = df[!, 19]
column_generation_solar = filter(!ismissing, column_generation_solar)

# Replication factor
factor = 1400

# Replicate the wind_speed array
column_generation_solar = repeat(column_generation_solar, factor)

println("Dataframe column_generation_solar has ", size(column_generation_solar, 1), " rows.")

Dataframe column_generation_solar has 49064400 rows.


___
### #1 Basic For Loop 


In [8]:
using Statistics

@show Threads.nthreads()

# Julia array with generation_solar data
column_generation_solar = Float64.(column_generation_solar)

# Alpha is the smoothing factor.
alpha = 0.7

# List to store execution times
number_of_executions = 10
execution_times = []
smoothed_value = column_generation_solar[1]

# Perform exponential smoothing on the data
for i in 1:number_of_executions
    start_time = time()

    # Initialize the smoothed value with the first data point
    smoothed_value = column_generation_solar[1]

    # Perform exponential smoothing on the data
    for j in 2:length(column_generation_solar)
        smoothed_value = alpha * column_generation_solar[j] + (1 - alpha) * smoothed_value
    end

    # Stop the timer
    end_time = time()
    push!(execution_times, end_time - start_time)
end

# Calculate the function time
function_time = mean(execution_times)

# Print the results
println("### Dataset generation_solar")
println("### #1 Basic For Loop \n")
println("The last smoothed value for generation_solar is: ", smoothed_value)
println("The function was executed in ", function_time, " seconds")


### Dataset generation_solar
### #1 Basic For Loop 

The last smoothed value for generation_solar is: 33.09027077319566
The function was executed in 11.085314512252808 seconds


___
### #2 Vectroized 

In [9]:
using Statistics

@show Threads.nthreads()

# Julia array with column_generation_solar data
column_generation_solar = Float64.(column_generation_solar)

# Alpha is the smoothing factor.
alpha = 0.7

# List to store execution times
number_of_executions = 10
execution_times = []
smoothed_value = column_generation_solar[1]

# Perform exponential smoothing on the data
for i in 1:number_of_executions
    start_time = time()

    # Perform exponential smoothing
    weights = alpha .* (1 - alpha) .^ (length(column_generation_solar)-1:-1:0)
    smoothed_values = cumsum(weights .* column_generation_solar) ./ cumsum(weights)
    smoothed_value = smoothed_values[end]

    # Stop the timer
    end_time = time()
    push!(execution_times, end_time - start_time)
end

# Calculate the function time
function_time = mean(execution_times)

# Print the results
println("### Dataset generation_solar")
println("### #2 Vectorized \n")
println("The last smoothed value for generation_solar is: ", smoothed_value)
println("The function was executed in ", function_time, " seconds")


### Dataset generation_solar
### #2 Vectorized 

The last smoothed value for generation_solar is: 33.09027077319566
The function was executed in 5.461846470832825 seconds


___
### #3 Parallelized Basic For Loop

In [10]:
using Statistics

Threads.nthreads() = 8
@show Threads.nthreads()

# Julia array with column_generation_solar data
column_generation_solar = Float64.(column_generation_solar)

# Alpha is the smoothing factor.
alpha = 0.7

# List to store execution times
number_of_executions = 10
execution_times = []
smoothed_value = column_generation_solar[1]

# Perform exponential smoothing on the data
for i in 1:number_of_executions
    start_time = time()

    # Initialize the smoothed value with the first data point
    smoothed_value = column_generation_solar[1]

    # Create an array to hold the intermediate smoothed values for each thread
    smoothed_values = fill(smoothed_value, length(column_generation_solar))

    # Perform exponential smoothing on the data
    Threads.@threads for j in 2:length(column_generation_solar)
        smoothed_values[j] = alpha * column_generation_solar[j] + (1 - alpha) * smoothed_values[j-1]
    end

    # The last smoothed value is the last element of the smoothed_values array
    smoothed_value = smoothed_values[end]

    # Stop the timer
    end_time = time()
    push!(execution_times, end_time - start_time)
end

# Calculate the function time
function_time = mean(execution_times)

# Print the results
println("### Dataset generation_solar")
println("### #3 Parallelized Basic For Loop \n")
println("The last smoothed value for generation_solar is: ", smoothed_value)
println("The function was executed in ", function_time, " seconds")


### Dataset generation_solar
### #3 Parallelized Basic For Loop 

The last smoothed value for generation_solar is: 33.09027077319566
The function was executed in 9.984675407409668 seconds


___
### #4 Parallelized Vectorized

In [22]:
using Statistics

Threads.nthreads() = 8
@show Threads.nthreads()

# Julia array with column_generation_solar data
column_generation_solar = Float64.(column_generation_solar)

# Alpha is the smoothing factor.
alpha = 0.7

# List to store execution times
number_of_executions = 10
execution_times = []
smoothed_value = column_generation_solar[1]

# Perform exponential smoothing on the data
for i in 1:number_of_executions
    start_time = time()

    # Perform exponential smoothing
    weights = alpha .* (1 - alpha) .^ (length(column_generation_solar)-1:-1:0)
    smoothed_values = cumsum(weights .* column_generation_solar) ./ cumsum(weights)
    smoothed_value = smoothed_values[end]

    # Stop the timer
    end_time = time()
    push!(execution_times, end_time - start_time)
end

# Calculate the function time
function_time = mean(execution_times)

# Print the results
println("### Dataset generation_solar")
println("### #2 Vectorized \n")
println("The last smoothed value for generation_solar is: ", smoothed_value)
println("The function was executed in ", function_time, " seconds")


### Dataset generation_solar
### #2 Vectorized 

The last smoothed value for generation_solar is: 33.09027077319566
The function was executed in 6.6144092082977295 seconds


___
___
## Dataset: Heart Rate Seconds

In [12]:
#read in the time series data 
using CSV
using DataFrames

df = CSV.read("/Users/niklas/Documents/GitHub/Uni/10_Masterarbeit/data_files/heartrate_seconds_merged.csv", DataFrame)

heart_rate_data = df[!, 3]
heart_rate_data = filter(!ismissing, heart_rate_data)

# Replication factor
factor = 90

# Replicate the wind_speed array
heart_rate_data = repeat(heart_rate_data, factor)

println("Dataframe heart_rate_data has ", size(heart_rate_data, 1), " rows.")

Dataframe heart_rate_data has 103921290 rows.


___
### #1 Basic Foor Loop 

In [13]:
using Statistics

@show Threads.nthreads()

# Julia array with heart_rate_data data
heart_rate_data = Float64.(heart_rate_data)

# Alpha is the smoothing factor.
alpha = 0.7

# List to store execution times
number_of_executions = 10
execution_times = []
smoothed_value = heart_rate_data[1]

# Perform exponential smoothing on the data
for i in 1:number_of_executions
    start_time = time()

    # Initialize the smoothed value with the first data point
    smoothed_value = heart_rate_data[1]

    # Perform exponential smoothing on the data
    for j in 2:length(heart_rate_data)
        smoothed_value = alpha * heart_rate_data[j] + (1 - alpha) * smoothed_value
    end

    # Stop the timer
    end_time = time()
    push!(execution_times, end_time - start_time)
end

# Calculate the function time
function_time = median(execution_times)

# Print the results
println("### Dataset heart_rate_data")
println("### #1 Basic For Loop \n")
println("The last smoothed value for heart_rate_data is: ", smoothed_value)
println("The function was executed in ", function_time, " seconds")


### Dataset heart_rate_data
### #1 Basic For Loop 

The last smoothed value for heart_rate_data is: 98.77752694571056
The function was executed in 24.440422415733337 seconds


___
### #2 Vectorized

In [14]:
using Statistics

@show Threads.nthreads()

# Julia array with heart_rate_data data
heart_rate_data = Float64.(heart_rate_data)

# Alpha is the smoothing factor.
alpha = 0.7

# List to store execution times
number_of_executions = 10
execution_times = []
smoothed_value = heart_rate_data[1]

# Perform exponential smoothing on the data
for i in 1:number_of_executions
    start_time = time()

    # Perform exponential smoothing
    weights = alpha .* (1 - alpha) .^ (length(heart_rate_data)-1:-1:0)
    smoothed_values = cumsum(weights .* heart_rate_data) ./ cumsum(weights)
    smoothed_value = smoothed_values[end]

    # Stop the timer
    end_time = time()
    push!(execution_times, end_time - start_time)
end

# Calculate the function time
function_time = median(execution_times)

# Print the results
println("### Dataset heart_rate_data")
println("### #2 Vectorized \n")
println("The last smoothed value for heart_rate_data is: ", smoothed_value)
println("The function was executed in ", function_time, " seconds")


### Dataset heart_rate_data
### #2 Vectorized 

The last smoothed value for heart_rate_data is: 98.77752694571055
The function was executed in 13.13746702671051 seconds


___
### #3 Parallelized Basic For Loop

In [15]:
using Statistics

Threads.nthreads() = 8
@show Threads.nthreads()

# Julia array with heart_rate_data data
heart_rate_data = Float64.(heart_rate_data)

# Alpha is the smoothing factor.
alpha = 0.7

# List to store execution times
number_of_executions = 1
execution_times = []
smoothed_value = heart_rate_data[1]

# Perform exponential smoothing on the data
for i in 1:number_of_executions
    start_time = time()

    # Initialize the smoothed value with the first data point
    smoothed_value = heart_rate_data[1]

    # Create an array to hold the intermediate smoothed values for each thread
    smoothed_values = fill(smoothed_value, length(heart_rate_data))

    # Perform exponential smoothing on the data
    Threads.@threads for j in 2:length(heart_rate_data)
        smoothed_values[j] = alpha * heart_rate_data[j] + (1 - alpha) * smoothed_values[j-1]
    end

    # The last smoothed value is the last element of the smoothed_values array
    smoothed_value = smoothed_values[end]

    # Stop the timer
    end_time = time()
    push!(execution_times, end_time - start_time)
end

# Calculate the function time
function_time = median(execution_times)

# Print the results
println("### Dataset heart_rate_data")
println("### #3 Parallelized Basic For Loop \n")
println("The last smoothed value for heart_rate_data is: ", smoothed_value)
println("The function was executed in ", function_time, " seconds")


### Dataset heart_rate_data
### #3 Parallelized Basic For Loop 

The last smoothed value for heart_rate_data is: 98.77752694571056
The function was executed in 27.48459792137146 seconds


___
### #4 Parallelized Vectorized

In [23]:
using Statistics

Threads.nthreads() = 8
@show Threads.nthreads()

# Julia array with heart_rate_data data
heart_rate_data = Float64.(heart_rate_data)

# Alpha is the smoothing factor.
alpha = 0.7

# List to store execution times
number_of_executions = 10
execution_times = []
smoothed_value = heart_rate_data[1]

# Perform exponential smoothing on the data
for i in 1:number_of_executions
    start_time = time()

    # Perform exponential smoothing
    weights = alpha .* (1 - alpha) .^ (length(heart_rate_data)-1:-1:0)
    smoothed_values = cumsum(weights .* heart_rate_data) ./ cumsum(weights)
    smoothed_value = smoothed_values[end]

    # Stop the timer
    end_time = time()
    push!(execution_times, end_time - start_time)
end

# Calculate the function time
function_time = median(execution_times)

# Print the results
println("### Dataset heart_rate_data")
println("### #4 Vectorized \n")
println("The last smoothed value for heart_rate_data is: ", smoothed_value)
println("The function was executed in ", function_time, " seconds")


### Dataset heart_rate_data
### #4 Vectorized 

The last smoothed value for heart_rate_data is: 98.77752694571055
The function was executed in 13.605377554893494 seconds


___
___